<center>

<h1> Using Prolog for Type Inference and Program Synthesis </h1>
</center>

$
\require{color}
\newcommand{\cr}[1]{{\color{\red}{#1}}}
\newcommand{\pair}[2]{\langle #1, #2 \rangle}
\newcommand{\fst}[1]{{\sf fst} ~#1}
\newcommand{\snd}[1]{{\sf snd} ~#1}
\newcommand{\inl}[2]{{\sf inl} ~[#1] ~#2}
\newcommand{\inr}[2]{{\sf inr} ~[#1] ~#2}
\newcommand{\case}[5]{{\sf case} ~#1~ {\sf of} ~{\sf inl}~#2 \Rightarrow #3 ~|~ {\sf inr}~#4 \Rightarrow #5}
\newcommand{\unitv}{{\tt (~)}}
\newcommand{\inferrule}[3]{\displaystyle{\frac{#1}{#2}~~{\small #3}}}
\newcommand{\infrl}[3]{\displaystyle{\frac{#1}{#2}}}
$

## This lecture

* Applications of Prolog
  + Type Inference for STLC
  + Program synthesis.

## Type Inference for STLC

Let us develop a type inference procedure for Simply Typed Lambda Calculus (STLC).

Recall the terms in STLC:

\\[
\begin{array}{rcll}
M,N & ≔ & x & \text{(variable)} \\
    & \mid & M~N & \text{(application)} \\
    & \mid & \lambda x:A.M & \text{(abstraction)}\\
    & \mid & \pair{M}{N} & \text{(pair)}\\
    & \mid & \fst{M} & \text{(project-1)}\\
    & \mid & \snd{M} & \text{(project-2)} \\
    & \mid & \unitv & \text{(unit)}
\end{array}
\\]

## STLC Typing Rules

\\[
\begin{array}{cc}
\inferrule{}{\Gamma,x:A \vdash x:A}{(var)} &
\inferrule{}{\Gamma \vdash \unitv : 1}{(unit)} \\ \\
\inferrule{\Gamma \vdash M : A \rightarrow B \quad \Gamma \vdash N : A}{\Gamma \vdash M~N : B}{(\rightarrow elim)} &
\inferrule{\Gamma,x:A \vdash M : B}{\Gamma \vdash \lambda x:A.M : A \rightarrow B}{(\rightarrow intro)} \\ \\
\inferrule{\Gamma \vdash M : A \times B}{\Gamma \vdash \fst{M} : A}{(\times ~elim1)} &
\inferrule{\Gamma \vdash M : A \times B}{\Gamma \vdash \snd{M} : B}{(\times ~elim2)} \\ \\
\end{array}
\\]

\\[
\inferrule{\Gamma \vdash M : A \quad \Gamma \vdash N : B}{\Gamma \vdash \pair{M}{N} : A \times B}{(\times ~intro)}
\\]

## Type Checking to Type Inference

* STLC rules are presented in a way that you can easily do type inference. 
* The standard Hindley-Milner type inference algorithm for STLC has to define the following things
  + Type schemes (types with variables in them)
  + Unification of type schemes
  + Substitution for variables in type schemes.
* Luckily Prolog provides all of these
  + Type schemes -> Prolog terms with variables, 
  + Unification -> Prolog unification
  + Substitution -> Prolog substitution. 

## Remove simple types & add polymorphism

* Since we have type schemes (variables in terms), we can infer **polymorphic types**!
* Rather than writing $\lambda x:A.M$, we just write $\lambda x.M$.
  + We will infer the principal (most general) type for $x$.
* We will also include integers, booleans, + and < on integers, if-then-else.

## Occurs check

We will enable occurs check so that the term $\lambda x.x~x$ (self-application) will be ill-typed.

In [ ]:
?- set_prolog_flag(occurs_check,true).

## Typing Judgement

We model the typing environment $\Gamma$ as a list of variable and type pairs. 

We implement the predicate `lookup/3` to lookup the type of a variable in the environment. 

In [ ]:
lookup([(X,A)|T],X,A).
lookup([(Y,_)|T],X,A) :-  X \== Y, lookup(T,X,A).

Observe that we are using ` X \== Y` which holds when `X` does not unify with `Y`.

## Typing rules

Next we encode the typing rules as they are specified in the STLC typing rules.

\\[
\begin{array}{cc}
\inferrule{}{\Gamma,x:A \vdash x:A}{(var)} &
\inferrule{}{\Gamma \vdash \unitv : 1}{(unit)} \\ \\
\inferrule{\Gamma \vdash M : A \rightarrow B \quad \Gamma \vdash N : A}{\Gamma \vdash M~N : B}{(\rightarrow elim)} &
\inferrule{\Gamma,x:A \vdash M : B}{\Gamma \vdash \lambda x:A.M : A \rightarrow B}{(\rightarrow intro)} \\ \\
\inferrule{\Gamma \vdash M : A \times B}{\Gamma \vdash \fst{M} : A}{(\times ~elim1)} &
\inferrule{\Gamma \vdash M : A \times B}{\Gamma \vdash \snd{M} : B}{(\times ~elim2)}
\end{array}
\\]

\\[
\inferrule{\Gamma \vdash M : A \quad \Gamma \vdash N : B}{\Gamma \vdash \pair{M}{N} : A \times B}{(\times ~intro)}
\\]

## STLC Terms

We will use functions for specifying different forms of STLC terms:

\\[
\begin{array}{rclll}
M,N & ≔ & x & \text{(variable)} & \texttt{var(x)}\\
 & \mid & M~N & \text{(application)} & \texttt{app(M,N)} \\
    & \mid & \lambda x.M & \text{(abstraction)} & \texttt{lam(x,M)}\\
    & \mid & \pair{M}{N} & \text{(pair)} & \texttt{pair(M,N)}\\
    & \mid & \fst{M} & \text{(project-1)} & \texttt{fst(M)}\\
    & \mid & \snd{M} & \text{(project-2)} & \texttt{snd(M)}\\
    & \mid & \unitv & \text{(unit)} & \texttt{u}
\end{array}
\\]

## Typing rules

Next we encode the typing rules as they are specified in the STLC typing rules. We will define the predicate `type(G,M,A)`, which corresponds to the typing judgement $\Gamma \vdash M : A$

In [ ]:
/* unit */     type(G,u,unit).                                          
/* -> elim */  type(G,app(M,N),B)           :- type(G,M,A -> B),type(G,N,A). 
/* -> intro */ type(G,lam(X,M),A -> B)      :- type([(X,A)|G],M,B).          
/* X elim1 */  type(G,fst(M),A)             :- type(G,M,A * B).              
/* X elim2 */  type(G,snd(M),B)             :- type(G,M,A * B).              
/* X intro */  type(G,pair(M,N),A * B)      :- type(G,M,A), type(G,N,B).     
/* var */      type(G,var(X),A)             :- lookup(G,X,A).                

## Typing rules

Add new typing rules for the additional terms and operators involving integers, booleans and if-then-else. 

In [ ]:
type(G, X, int)        :- integer(X).
type(G, true, bool).
type(G, false, bool).
type(G, A + B, int)    :- type(G,A,int), type(G,B,int).
type(G, A - B, int)    :- type(G,A,int), type(G,B,int).
type(G, A < B, bool)   :- type(G,A,int), type(G,B,int).
type(G, ite(A,B,C), T) :- type(G,A,bool), type(G,B,T), type(G,C,T).

In [ ]:
type(Term,Type) :- type([],Term,Type).

## Type inference

Now we can infer the type of programs written in STLC.

What is the type of $1+2$?

In [ ]:
?- type(1+2,X).

what is the type of $\lambda x.\lambda y. \text{if } x < y \text{ then } x+y \text{ else } x-y$?

In [ ]:
?- X = var(x), Y = var(y), type(lam(x,lam(y,ite(X < Y,X+Y,X-Y))),T).

It is `int -> int -> int`.

## Type inference

We can also infer polymorphic types.

What is the type of $\lambda x.\text{fst}(x)+1$?

In [ ]:
?- X = var(x), type(lam(x,fst(X)+1),T).

It is `int * 'a -> int`.

## Type inference

What is the type of $\lambda f. \lambda x.f~x$?

In [ ]:
?- F = var(f), X = var(x), type(lam(f,lam(x,app(F,X))),T).

It is `('a -> 'b) -> ('a -> 'b)` or equivalently `('a -> 'b) -> 'a -> 'b`

## Type Inference

* We cannot infer types for every program.
  + such programs do not have a valid STLC type.
  
What is the type of $\lambda x.x~x$?

In [ ]:
?- X = var(x), type(lam(x,app(X,X)),T).

What is the type of `if true then 0 else false`?

In [ ]:
?- type(ite(true,0,false),T).

## Program synthesis

* Program synthesis is generating programs according to a given specification. 
* Our specifications are types!
  + A query of the form `type(X,A)` can find lambda terms `X` whose type is `A`.
* Let's generate lambda terms that correspond to a particular type. 
  + We will bound the size of the lambda term. 
  + Otherwise, Prolog starts to explore down infinite paths
    * programs have no bounded length and Prolog uses DFS.
* Let's use the depth of the AST in order to iteratively search starting from depth of 0.

## Bounded predecessor

Defines the predecessor for numbers >= 0.

In [ ]:
pred(N,P) :- N >= 0, P is N - 1.

## Add depth to the type checking rules

In [ ]:
type(_,u,unit,D) :- 
  pred(D,_).
/* OLD RULE for -> elim ==> type(G,app(M,N),B) :- type(G,M,A -> B),type(G,N,A). */
type(G,app(M,N),B,D) :- 
  pred(D,DD), type(G,M,A -> B,DD),type(G,N,A,DD).             
type(G,lam(X,M),A -> B, D) :-                                                  
  pred(D,DD), type([(X,A)|G],M,B, DD).                                              
type(G,fst(M),A,D) :-                                                               
  pred(D,DD), type(G,M,A * _,DD).                                                   
type(G,snd(M),B,D) :-                                                               
  pred(D,DD), type(G,M,_ * B,DD).                                                   
type(G,pair(M,N),A * B,D) :-                                                        
  pred(D,DD), type(G,M,A,DD), type(G,N,B,DD).                                       
type(G,var(X),A,D) :-                                                               
  pred(D,_), lookup(G,X,A).                                                         

## Add depth to the type checking rules.

In [ ]:
type(_,X,int,D) :-                                                                  
  pred(D,_), integer(X).                                                            
type(_,D,int,D) :-                                                                  
  pred(D,_).                                                                        
type(_,true,bool,D) :-                                                              
  pred(D,_).                                                                        
type(_,false,bool,D) :-                                                             
  pred(D,_).                                                                        
type(G,A + B,int,D) :-                                                              
  pred(D,DD), type(G,A,int,DD), type(G,B,int,DD).                                   
type(G,A < B,bool,D) :-                                                             
  pred(D,DD), type(G,A,int,DD), type(G,B,int,DD).                                   
type(G,ite(A,B,C),T,D) :-                                                           
  pred(D,DD), type(G,A,bool,DD), type(G,B,T,DD), type(G,C,T,DD).

## Generate depth

Define a function `gen/3` which generates all values within given bounds (inclusive).


In [ ]:
gen(S,E,S).
gen(S,E,P) :- S < E, S2 is S+1, gen(S2,E,P).

In [ ]:
?- gen(0,5,X).

## Iteratively search for candidate programs

In [ ]:
synthesize(T,P) :-                                                                  
  gen(0,10,D), type([],P,T,D).     

## Synthesis

Get me those programs whose type is `int`.

In [ ]:
?- synthesize(int,P) {1000}.

## Synthesis

Let's ask for something more interesting. 

Get me the program whose type is `A * B -> A`. One answer: $\lambda p.\fst{p}$.

In [ ]:
?- synthesize((A*B)->A,P).

## Synthesis

* Valid but uninteresting. 
+ We are interested in "Most general program(s)"; dual of most general type during inference.
  - The program that doesn't specialise `A` and `B`. 
+ In other words, types must remain polymorphic.

## Synthesize

Get me the program whose type is `A * B -> A`, where `A` and `B` remain polymorphic.

In [ ]:
?- synthesize((A*B)->A,P), var(A), var(B) {1}.

That's the program we are looking for: $\lambda p.\fst{p}$.

Aside: `var/1` is a built-in Prolog predicate which is `true` if its input argument is a variable.

## Synthesize

Get me the program whose type is `A*(B*C) -> B`, where `A`,`B`,`C` remain polymorphic.

In [ ]:
?- synthesize((A*(B*C))->B,P), var(A), var(B), var(C) {1}.

`A`, `B`, `C` should not unify.

In [ ]:
?- synthesize((A*(B*C))->B,P), var(A), var(B), var(C), dif(A,B) {1}.

That's the program we are looking for: $\lambda p.\fst({\snd{p}})$.